In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [172]:
"""
Created on Mon Feb 18 20:30:48 2019

@author: jschmidt
"""

import os, shutil, random

import pandas as pd
import sys
import gdal
import imp
import numpy as np


import scripts.windturbines.functions_pattern_recognition as fpr
imp.reload(fpr)
from scripts.windturbines.functions_pattern_recognition import get_param
from scripts.windturbines.functions_pattern_recognition import cop_predict
from scripts.windturbines.functions_pattern_recognition import check_image
from scripts.windturbines.functions_pattern_recognition import read_params

In [173]:
COUNTRY = "US"

categories = ["small", "middle", "large"]

train_dir_small = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_SMALL_TRAIN")
test_dir_small = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_SMALL_TEST")
validation_dir_small = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_SMALL_VALIDATION")

train_dir_middle = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_MIDDLE_TRAIN")
test_dir_middle = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_MIDDLE_TEST")
validation_dir_middle = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_MIDDLE_VALIDATION")

train_dir_large = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_LARGE_TRAIN")
test_dir_large = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_LARGE_TEST")
validation_dir_large = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_LARGE_VALIDATION")

train_dirs = [train_dir_small, train_dir_middle, train_dir_large]
test_dirs = [test_dir_small, test_dir_middle, test_dir_large]
val_dirs = [validation_dir_small, validation_dir_middle, validation_dir_large]


#### delete directories if exist
#### create if not exist

shutil.rmtree(train_dir_small,ignore_errors=True)
shutil.rmtree(test_dir_small,ignore_errors=True)
shutil.rmtree(validation_dir_small,ignore_errors=True)

shutil.rmtree(train_dir_middle,ignore_errors=True)
shutil.rmtree(test_dir_middle,ignore_errors=True)
shutil.rmtree(validation_dir_middle,ignore_errors=True)

shutil.rmtree(train_dir_large,ignore_errors=True)
shutil.rmtree(test_dir_large,ignore_errors=True)
shutil.rmtree(validation_dir_large,ignore_errors=True)


os.makedirs(train_dir_small)
os.makedirs(test_dir_small)
os.makedirs(validation_dir_small)

os.makedirs(train_dir_middle)
os.makedirs(test_dir_middle)
os.makedirs(validation_dir_middle)

os.makedirs(train_dir_large)
os.makedirs(test_dir_large)
os.makedirs(validation_dir_large)

src_dir_tb = get_param(COUNTRY,"PATH_RAW_IMAGES_TURBINES")






In [174]:
turbines_us = pd.read_csv(get_param("US","FILE_TURBINE_LOCATIONS"))

predictions_cs = get_param("US",
                            "PATH_RAW_IMAGES_TURBINES")

turbines_pred = pd.read_feather(predictions_cs + "all_predictions.feather")

turbines_us = turbines_us[turbines_pred['prediction'] > 0.99]

In [175]:
turbines_us.shape

(47665, 27)

In [176]:
numb_of_elements = [np.sum(turbines_us['t_hh']<80),
                   np.sum((turbines_us['t_hh']>80) & (turbines_us['t_hh']<100)),
                   np.sum(turbines_us['t_hh']>100)]

In [168]:
nmb_samples = min(numb_of_elements)/numb_of_elements
turbines_us['id'] = np.arange(len(turbines_us))



In [157]:
nmb_samples_all = min(numb_of_elements)

In [169]:
turbines_us_small = turbines_us[turbines_us['t_hh'] < 60].sample(frac = nmb_samples[0])
turbines_us_middle = turbines_us[(turbines_us['t_hh']>60) & (turbines_us['t_hh']<90)].sample(frac = nmb_samples[1])
turbines_us_large = turbines_us[turbines_us['t_hh']>90].sample(frac = nmb_samples[2])

share_train=round(0.7*nmb_samples_all)
share_validate=round(0.85*nmb_samples_all)

turbines = [turbines_us_small, turbines_us_middle, turbines_us_large]


In [170]:
nmb_samples_all

2399

In [171]:
for i in range(0,3):
    
    cnt = 0
    
    print(categories[i])
    
    for j in range(0, nmb_samples_all):
        cnt += 1
        
        
        #print(file)
        file_s = str(turbines[i].iloc[[j]]['id'].values[0] + 1) + '.tif'
        file_d = str(turbines[i].iloc[[j]]['id'].values[0] + 1) + '.png'
        
        #print(file)
        
        src = os.path.join(src_dir_tb, file_s)
        
        
        
        if(cnt % 1000 == 0):
            print(cnt)
            print(src)
            
        #print(src)
        
        
        
        if(os.path.isfile(src)):
            
            dst = ""
        
            if(cnt < share_train):
           
                dst = os.path.join(train_dirs[i], file_d)
            
            if(cnt>share_train and cnt<share_validate):
           
                dst = os.path.join(val_dirs[i], file_d)
            
            if(cnt>share_validate):
        
                dst = os.path.join(test_dirs[i],file_d)
            
            #print(src + " " + dst)
            
            fpr.copyfile_to_png(src, dst)
            
    #### copy no-turbine images

small


IndexError: positional indexers are out-of-bounds

In [91]:
for i in range(0,3):
    
    print('total training turbine images ' + categories[i], len(os.listdir(train_dirs[i])))
    print('total testing turbine images ' + categories[i], len(os.listdir(test_dirs[i])))
    print('total validation turbine images ' + categories[i], len(os.listdir(val_dirs[i])))


total training turbine images small 0
total testing turbine images small 0
total validation turbine images small 0
total training turbine images middle 0
total testing turbine images middle 0
total validation turbine images middle 0
total training turbine images large 0
total testing turbine images large 0
total validation turbine images large 0


16502